# Métodos necesarios para los procesos 

In [2]:
import os
import pathlib
import requests
import math

import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import IPython
import numpy as np
import seaborn as sns
from pydub import AudioSegment, effects

import noisereduce as nr
import librosa
import soundfile as sf

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_hub as hub


seed = 1069

np.random.seed = seed

#Convierto los archivos que originalmente son mp3 a wav para decodificarlos adecuadamente
def convert_to_mp3_to_wav(ruta_archivo):

        try:
            # Cargar el archivo de audio MP3
            audio_mp3 = AudioSegment.from_mp3(ruta_archivo)

            # Convertir el audio a formato WAV
            audio_wav = audio_mp3.export(ruta_archivo, format="wav")
        except Exception:
            print("")

def duracion_a_segundos(duracion):
    # Dividir el string en horas, minutos y segundos
    minutos, segundos = duracion.split(":")
    
    # Convertir a enteros
    minutos = int(minutos)
    segundos = int(segundos)
    
    # Calcular el total de segundos
    total_segundos = minutos * 60 + segundos
    
    return total_segundos


#Obtengo las muestras disponibles sobre ese pájaro
def buscar_registros_xeno_canto(nombre_cientifico):
    url_busqueda = f"https://www.xeno-canto.org/api/2/recordings?query={nombre_cientifico.replace(' ', '%20')}"
    
    print(f"https://www.xeno-canto.org/api/2/recordings?query={nombre_cientifico.replace(' ', '%20')}")
    # Realizar la solicitud GET para buscar registros de sonido
    respuesta = requests.get(url_busqueda)
    

    # Verificar si la solicitud fue exitosa
    if respuesta.status_code == 200:
        datos = respuesta.json()
        # ID de los audios, licencia y autor
        return ([registro['id'] for registro in datos['recordings'] if registro['q'] == "A" and len(registro['also']) == 0 and not "by-nc-nd" in registro['lic'] and not "by-nd" in registro['lic']], [lic['lic'] for lic in datos['recordings'] if lic['q'] == "A" and len(lic['also']) == 0 and not "by-nc-nd" in lic['lic'] and not "by-nd" in lic['lic']], [rec['rec'] for rec in datos['recordings'] if rec['q'] == "A" and len(rec['also']) == 0 and not "by-nc-nd" in rec['lic'] and not "by-nd" in rec['lic']])
    else:
        print("Error al buscar registros de sonido.")
        return []

def descargar_audio_xeno_canto(id_registro, directorio_destino, is_train):
    url_base = "https://www.xeno-canto.org/"
    url_registro = f"{url_base}{id_registro}/download"
    
    
    # Realizar la solicitud GET para descargar el archivo de audio
    respuesta = requests.get(url_registro)
    
    # Verificar si la solicitud fue exitosa
    if respuesta.status_code == 200:
        # Obtener el nombre del archivo desde la URL de descarga
        nombre_archivo = id_registro + ".wav"
        
        # Ruta completa para guardar el archivo
        ruta_archivo = os.path.join(directorio_destino, nombre_archivo)
        
        # Escribir el contenido del audio en el archivo
        with open(ruta_archivo, "wb") as archivo:
            archivo.write(respuesta.content)

        convert_to_mp3_to_wav(ruta_archivo)

        nombre_archivo = nombre_archivo.replace(".wav", "")

        create_spect_reduce_noise(ruta_archivo, directorio_destino, nombre_archivo, is_train)
        
        #print("¡Descarga completa!")
        print("Archivo guardado en:", ruta_archivo)
    else:
        print("Error al descargar el archivo.")

def descargar_audio_pajaro_por_nombre_cientifico(nombre_cientifico, directorio_destino, numero_ficheros_train, prob_ficheros_test, prob_ficheros_validacion):
    # Buscar registros de sonido del pájaro
    data = buscar_registros_xeno_canto(nombre_cientifico)
    ids_registros = data[0]


    
    if ids_registros:
        print(f"Encontrados {len(ids_registros)} registros de sonido para {nombre_cientifico}. Descargando archivos de audio...")
        
        if (numero_ficheros_train > len(ids_registros)):
            print("No existen tantos fichero para descargar, se descargarán todos los posibles")
            numero_ficheros_train = len(ids_registros)
            
        numero_ficheros_test = round(numero_ficheros_train * prob_ficheros_test)
        numero_ficheros_validacion = round(numero_ficheros_train * prob_ficheros_validacion)
        numero_ficheros_train -= round(numero_ficheros_test + numero_ficheros_validacion)
        


        #train
        # Descargar archivos de audio que quiero y de forma aleatoria para evitar sesgo
        for i in range(0, numero_ficheros_train):
            #Genero numero aleatorio para seleccionar una muestra aleatoria
            number_generated = np.random.randint(0, len(ids_registros))
            #print("Numero aleatorio: " + str(number_generated))
            #Saco la muestra
            registro = ids_registros[number_generated]
            #print("Fichero a descargar: " + str(registro))
            #Escribo la licencia y autor del registro
            str_fichero = "ID_registro: " + str(registro) + " Licencia: " + str(data[1][number_generated]) + " Autor: " + str(data[2][number_generated])
            guardar_strings_en_archivo(str_fichero, "Reconocimiento.txt")
            #Elimino la muestra para que no vuelva a salir
            ids_registros.pop(number_generated)
            data[1].pop(number_generated)
            data[2].pop(number_generated)
            descargar_audio_xeno_canto(registro, directorio_destino, True)

        #validacion
        directorio_destino_validacion = directorio_destino.replace("train", "validation")

        if not os.path.exists(directorio_destino_validacion):
            os.makedirs(directorio_destino_validacion)

        for i in range(0, numero_ficheros_validacion):
            #Genero numero aleatorio para seleccionar una muestra aleatoria
            number_generated = np.random.randint(0, len(ids_registros))
            #print("Numero aleatorio: " + str(number_generated))
            #Saco la muestra
            registro = ids_registros[number_generated]
            #print("Fichero a descargar: " + str(registro))
            #Escribo la licencia y autor del registro
            str_fichero = "ID_registro: " + str(registro) + " Licencia: " + str(data[1][number_generated]) + " Autor: " + str(data[2][number_generated])
            guardar_strings_en_archivo(str_fichero, "Reconocimiento.txt")
            #Elimino la muestra para que no vuelva a salir
            ids_registros.pop(number_generated)
            data[1].pop(number_generated)
            data[2].pop(number_generated)
            descargar_audio_xeno_canto(registro, directorio_destino_validacion, False)    
        
        #test
        directorio_destino_test = directorio_destino.replace("train", "test")

        if not os.path.exists(directorio_destino_test):
            os.makedirs(directorio_destino_test)

        for i in range(0, numero_ficheros_test):
            #Genero numero aleatorio para seleccionar una muestra aleatoria
            number_generated = np.random.randint(0, len(ids_registros))
            #print("Numero aleatorio: " + str(number_generated))
            #Saco la muestra
            registro = ids_registros[number_generated]
            #print("Fichero a descargar: " + str(registro))
            #Escribo la licencia y autor del registro
            str_fichero = "ID_registro: " + str(registro) + " Licencia: " + str(data[1][number_generated]) + " Autor: " + str(data[2][number_generated])
            guardar_strings_en_archivo(str_fichero, "Reconocimiento.txt")
            #Elimino la muestra para que no vuelva a salir
            ids_registros.pop(number_generated)
            data[1].pop(number_generated)
            data[2].pop(number_generated)
            descargar_audio_xeno_canto(registro, directorio_destino_test, False)       
    else:
        print(f"No se encontraron registros de sonido para {nombre_cientifico}.")

def guardar_strings_en_archivo(strings, nombre_archivo):
    try:
        with open(nombre_archivo, 'a', encoding="utf-8") as archivo:
            if (strings is not None or strings != ""):
                archivo.writelines(strings + '\n')
            
    except IOError:
        print(f'Error: No se pudo abrir o escribir en el archivo "{nombre_archivo}".')
    
def reduce_sampling_rate(input_file_path, output_file, target_sr):

    # Cargar el archivo de audio
    y, sr = librosa.load(input_file_path, sr=None)

    # Centrando la onda en cero
    y_centrado = y - np.mean(y)

    y_resampled = librosa.util.normalize(y_centrado)

    # Reducir la tasa de muestreo
    y_resampled = librosa.resample(y_resampled, orig_sr=sr, target_sr=target_sr)

    # Guardar el archivo de audio con la nueva tasa de muestreo
    sf.write(output_file, y_resampled, target_sr)

def reduce_noise(audio_file_path, output_file):

    # Cargar el archivo de audio
    data, sample_rate = librosa.load(audio_file_path)

    # Reducir el ruido del audio
    reduced_noise = nr.reduce_noise(y=data, sr=sample_rate)

    # Guardar el archivo de audio filtrado
    sf.write(output_file, reduced_noise, sample_rate)

def get_mel_spectogram(audio_file_path, output_file, output_file_split):
    audio, sr = librosa.load(audio_file_path)

    n_fft = librosa.stft(audio).shape[0]

    espectrograma = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft)
    plt.figure(figsize=(32, 32))
    
    # Convertir el espectrograma a decibeles (dB)
    espectrograma_dB = librosa.power_to_db(espectrograma, ref=np.max)

    librosa.display.specshow(espectrograma_dB, sr=sr, n_fft=4096)
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close('all')

    #Empiezo a trocear
    index_trim = librosa.effects.split(audio, top_db=10, frame_length=8192, hop_length=4096)

    i = 0
    for index in index_trim:
        # Calcular el espectrograma mel
        espectrograma = librosa.feature.melspectrogram(y=audio[index[0]:index[1]], sr=sr)

        # Convertir el espectrograma a decibeles (dB)
        espectrograma_dB = librosa.power_to_db(espectrograma, ref=np.max)

        # Visualizar el espectrograma mel
        plt.figure(figsize=(32, 32))
        librosa.display.specshow(espectrograma_dB, sr=sr)
        plt.savefig(output_file_split.replace(".jpg", "_"+str(i)+".jpg"), bbox_inches='tight', pad_inches=0)
        plt.close('all')
        i+=1


def create_spect_reduce_noise(wav_audio_path, output_path, nombre_archivo, is_train):

    audio = tf.io.read_file(wav_audio_path)


    fig, axes = plt.subplots(1, figsize=(20, 20))

    
    reduce_sampling_rate_output_path = output_path.replace("data_set_birds", "reduce_sampling_rate_set_birds") + "\\"

    if not os.path.exists(reduce_sampling_rate_output_path):
        os.makedirs(reduce_sampling_rate_output_path)

    reduce_sampling_rate(wav_audio_path, reduce_sampling_rate_output_path + nombre_archivo + ".wav", 16000)

    reduce_noise_output_path = output_path.replace("data_set_birds", "reduce_noise_set_birds") + "\\"

    if not os.path.exists(reduce_noise_output_path):
        os.makedirs(reduce_noise_output_path)

    reduce_noise(reduce_sampling_rate_output_path + nombre_archivo + ".wav", reduce_noise_output_path + nombre_archivo + ".wav")

    #spect_sample = get_spectrogram(data, sample_rate)

    #spec_reduce_noise_output_path = output_path.replace("data_set_birds", "spec_reduce_noise_set_birds") + "\\"

    #if not os.path.exists(spec_reduce_noise_output_path):
    #    os.makedirs(spec_reduce_noise_output_path)

    #save_spectrogram(spect_sample, axes, spec_reduce_noise_output_path + nombre_archivo + ".jpg")

    melspectogram_reduce_noise_output_path_split = output_path.replace("data_set_birds", "melspectogram_reduce_noise_set_birds_split") + "\\"

    if not os.path.exists(melspectogram_reduce_noise_output_path_split):
        os.makedirs(melspectogram_reduce_noise_output_path_split)
    
    melspectogram_reduce_noise_output_path = output_path.replace("data_set_birds", "melspectogram_reduce_noise_set_birds") + "\\"

    if not os.path.exists(melspectogram_reduce_noise_output_path):
        os.makedirs(melspectogram_reduce_noise_output_path)

    get_mel_spectogram(reduce_noise_output_path + nombre_archivo + ".wav", melspectogram_reduce_noise_output_path + nombre_archivo + ".jpg", melspectogram_reduce_noise_output_path_split + nombre_archivo + ".jpg")

    if not os.path.exists(output_path.replace("data_set_birds", "data_augmentation_shift_birds")):
        os.makedirs(output_path.replace("data_set_birds", "data_augmentation_shift_birds"))

    if not os.path.exists(output_path.replace("data_set_birds", "data_augmentation_pitch_birds")):
        os.makedirs(output_path.replace("data_set_birds", "data_augmentation_pitch_birds"))

    output_file_data_augmentation = output_path.replace("data_set_birds", "data_augmentation_birds") + "\\"
    mel_outpup_data_augmentation = output_path.replace("data_set_birds", "data_augmentation_birds_split") + "\\"

    if not os.path.exists(output_file_data_augmentation):
        os.makedirs(output_file_data_augmentation)

    if not os.path.exists(mel_outpup_data_augmentation):
        os.makedirs(mel_outpup_data_augmentation)
    
    if (is_train):
        data_augmentation(reduce_noise_output_path + nombre_archivo + ".wav", output_file_data_augmentation + nombre_archivo + ".jpg", mel_outpup_data_augmentation + nombre_archivo + ".jpg")


def data_augmentation(audio_file_path, output_file, mel_outpup_file):
    # Cargar el archivo de audio
    data, sample_rate = librosa.load(audio_file_path)

    data_shift = manipulate_shift(data, sample_rate, 3, 'right')

    sf.write(audio_file_path.replace("reduce_noise_set_birds", "data_augmentation_shift_birds"), data_shift, sample_rate)


    get_mel_spectogram(audio_file_path.replace("reduce_noise_set_birds", "data_augmentation_shift_birds"), output_file.replace(".jpg", "shift.jpg"), mel_outpup_file.replace(".jpg", "shift.jpg"))

    data_pitch = maniputalete_pitch(data, sample_rate, 0.3)

    sf.write(audio_file_path.replace("reduce_noise_set_birds", "data_augmentation_pitch_birds"), data_pitch, sample_rate)

    get_mel_spectogram(audio_file_path.replace("reduce_noise_set_birds", "data_augmentation_pitch_birds"), output_file.replace(".jpg", "pitch.jpg"), mel_outpup_file.replace(".jpg", "pitch.jpg"))





def maniputalete_pitch(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=4)

def manipulate_shift(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

def crear_conjuntos_datos(nombres_cientificos, ficheros_descargar, distribución_test, distribucion_validation):
    plt.ioff()

    for i in nombres_cientificos:
        nombre_cientifico = i

        # Crear el directorio si no existe
        directorio_destino = "train\\data_set_birds" + "\\" + nombre_cientifico.replace(" ", "_")

        if not os.path.exists(directorio_destino):
            os.makedirs(directorio_destino)

        # Llamar a la función para descargar los archivos de audio del pájaro
        descargar_audio_pajaro_por_nombre_cientifico(nombre_cientifico, directorio_destino, ficheros_descargar, distribución_test, distribucion_validation)

        plt.close('all')



# Método para generar los conjuntos de datos

In [ ]:
nombres_cientificos = [
    "Alauda arvensis",          # Alondra común
    "Carduelis carduelis",      # Jilguero
    "Passer domesticus",        # Gorrión común
    "Turdus merula",            # Mirlo común
    "Erithacus rubecula",       # Petirrojo europeo
    "Upupa epops",              # Abubilla
    "Streptopelia turtur",      # Tórtola europea
    "Hirundo rustica",          # Golondrina común
    "Falco tinnunculus",        # Cernícalo vulgar
    "Milvus migrans",           # Milano negro
    "Cyanistes caeruleus",      # Herrerillo común
    "Parus major",              # Carbonero común
    "Sturnus vulgaris",         # Estornino pinto
    "Phoenicurus ochruros",     # Colirrojo tizón
    "Lanius meridionalis",      # Alcaudón meridional
    "Pica pica",                # Urraca
    "Anthus pratensis",         # Bisbita pratense
    "Saxicola rubicola",        # Tarabilla común
    "Cuculus canorus",          # Cuco común
    "Corvus corax",             # Cuervo grande
    "Asio otus",                # Búho chico
    "Lullula arborea",          # Totovía
    "Pyrrhocorax pyrrhocorax",  # Chova piquirroja
    "Circus pygargus",          # Aguilucho cenizo
    "Accipiter nisus",          # Gavilán común
    "Columba livia",            # Paloma bravía
    "Cettia cetti",             # Ruiseñor bastardo
    "Apus apus",                # Vencejo común
    "Luscinia megarhynchos",    # Ruiseñor común
    "Aegithalos caudatus",      # Mito
    "Regulus regulus",          # Reyezuelo sencillo
    "Delichon urbicum",         # Avión común
    "Muscicapa striata",        # Papamoscas gris
    "Ficedula hypoleuca",       # Papamoscas cerrojillo
    "Troglodytes troglodytes",  # Chochín común
    "Oriolus oriolus",          # Oropéndola
    "Phylloscopus collybita",   # Mosquitero común
    "Emberiza cirlus",          # Escribano soteño
    "Galerida cristata",        # Cogujada común
    "Merops apiaster",          # Abejaruco europeo
    "Ardea cinerea",            # Garza real
    "Pernis apivorus",          # Abejero europeo
    "Scolopax rusticola",       # Chocha perdiz
    "Anas platyrhynchos",       # Ánade real
    "Buteo buteo",              # Busardo ratonero
    "Tachymarptis melba",       # Vencejo real
    "Pluvialis apricaria",      # Chorlito dorado europeo
    "Vanellus vanellus",        # Avefría europea
    "Alectoris rufa",           # Perdiz roja
    "Charadrius dubius"         # Chorlitejo chico
]

print(len(nombres_cientificos))

crear_conjuntos_datos(nombres_cientificos, 125, 0.1, 0.1)